## The goal of this notebook is to explore Hershey promotion data and experiment with DSI

In [24]:
# Import package
import pandas as pd
import numpy as np

import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

## Process raw data

In [25]:
def create_output_folder(base_path):
    """Create a timestamped output folder."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_folder = os.path.join(base_path, f"output_{timestamp}")
    os.makedirs(output_folder, exist_ok=True)
    return output_folder


def load_and_preprocess_data(filepath):
    """Load data from a CSV file and perform initial preprocessing."""
    df = pd.read_csv(filepath)
    df['week_start_date'] = pd.to_datetime(df['week_start_date'])
    # replace all missing value with 0 if data type is numeric
    df.fillna(0, inplace=True)
    return df

def aggregate_data(df, groupby_cols, sum_columns, first_non_missing_columns):
    """Aggregate data by specified columns with sum and first non-missing value rules."""
    agg_dict = {col: 'sum' for col in sum_columns}
    agg_dict.update({col: 'first' for col in first_non_missing_columns})
    aggregated_df = df.groupby(groupby_cols).agg(agg_dict).reset_index()

    # Replace promotion_type with NA if on_promotion is False
    aggregated_df.loc[aggregated_df['on_promotion'] == False, 'promotion_type'] = np.nan

    # Create dummy variables for promotion_type
    aggregated_df = pd.get_dummies(aggregated_df, columns=['promotion_type'], dummy_na=True)
    return aggregated_df

In [26]:
# Update the input file location to use Linux-style path
filepath = "../../data/hershey_promo_impact/input/shipment_hist_weekly_with_promo_features.csv"

# Update the output file location to use Linux-style path
base_output_path ="../../data/hershey_promo_impact/output"
output_folder = create_output_folder(base_output_path)
print(f"All outputs will be saved to {output_folder}")

groupby_cols = ['sales_org', 'dp_cust', 'dmd_item_10', 'week_start_date']
sum_columns = ['qty', 'total_expense_trade', 'forecast_cot_spend',  'po_qty_week_1', 'po_qty_week_2', 'po_qty_week_3', 'po_qty_week_4', 'po_qty_week_8']
first_non_missing_columns = [
    'descr', 'division', 'franchise', 'subbrand', 'brand', 'packtype',
    'ppg_family', 'ppg', 'season', 'sales_status', 'on_promotion',
    'promotion_type', 'total_expense_trade', 'forecast_cot_spend', 'price_sch3'
]

df = load_and_preprocess_data(filepath)
print('finished loading data')

All outputs will be saved to ../../data/hershey_promo_impact/output\output_20250121_194227
finished loading data


In [29]:
aggregated_df = aggregate_data(df, groupby_cols, sum_columns, first_non_missing_columns)
print('finished aggregating data')
aggregated_df['unit_id'] = aggregated_df['sales_org'].astype(str) + '_' + aggregated_df['dp_cust'].astype(str) + '_' + aggregated_df['dmd_item_10'].astype(str)
aggregated_df.columns

finished aggregating data


Index(['sales_org', 'dp_cust', 'dmd_item_10', 'week_start_date', 'qty',
       'total_expense_trade', 'forecast_cot_spend', 'po_qty_week_1',
       'po_qty_week_2', 'po_qty_week_3', 'po_qty_week_4', 'po_qty_week_8',
       'descr', 'division', 'franchise', 'subbrand', 'brand', 'packtype',
       'ppg_family', 'ppg', 'season', 'sales_status', 'on_promotion',
       'price_sch3', 'promotion_type_Corporate Promotions',
       'promotion_type_Correction', 'promotion_type_EDLC',
       'promotion_type_EDLP', 'promotion_type_Hi Lo',
       'promotion_type_Miscellaneous', 'promotion_type_nan', 'unit_id'],
      dtype='object')

In [31]:
aggregated_df['log_qty'] = np.log1p(aggregated_df['qty']+1)
# Then define x_base_columns with the newly created columns
x_base_columns = [
    'unit_id',
    'week_start_date',
    'promotion_type_EDLP',
    'brand',
    'log_qty'
]
wdf = aggregated_df[x_base_columns]
wdf

,unit_id,week_start_date,promotion_type_EDLP,brand,log_qty
0,US04_6902037_5571200,2021-09-13,False,0,0.693147
1,US04_6902037_5571200,2021-09-20,False,0,0.693147
2,US04_6902037_5571200,2021-09-27,False,0,0.693147
3,US04_6902037_5571200,2021-10-04,False,0,0.693147
4,US04_6902037_5571200,2021-10-11,False,0,0.693147
...,...,...,...,...,...
363307,USBX_6902053_5025100443,2025-09-29,False,SKINNYPOP POPCORN,0.693147
363308,USBX_6902053_5025100443,2025-10-06,False,SKINNYPOP POPCORN,0.693147
363309,USBX_6902053_5025100443,2025-10-13,False,SKINNYPOP POPCORN,0.693147
363310,USBX_6902053_5025100443,2025-10-20,False,SKINNYPOP POPCORN,0.693147


## EDA

In [33]:
wdf.describe()

,week_start_date,log_qty
count,363312,363312.000000
mean,2023-10-05 11:59:59.999999744,1.823592
min,2021-09-13 00:00:00,0.693147
25%,2022-09-24 06:00:00,0.693147
50%,2023-10-05 12:00:00,0.693147
75%,2024-10-15 18:00:00,0.693147
max,2025-10-27 00:00:00,13.393919
std,NaN,2.328889


## Transform data to cross sectional

In [111]:
def transform_to_cross_sectional(df, 
                                 relative_week,
                                 treatment_var = 'promotion_type_EDLP', 
                                 Y_var = 'log_qty'):

    # Filter the dataframe for the given relative_week
    df_filtered = df[df['week_start_date'] == relative_week]

    # Rename Y and treatment columns to reflect the relative week
    df_filtered = df_filtered.rename(columns={
        Y_var: 'y',
        treatment_var: 'treatment'
    })

    # Merge the lagged Y values for other weeks
    df = df[df['week_start_date'] <= relative_week]
    lagged_Y = df.pivot(index='unit_id', columns='week_start_date', values=Y_var)
    lagged_Y.columns = [f'Y_lag_{col}' for col in lagged_Y.columns]

    # Merge lagged Y with the filtered data
    result = pd.merge(df_filtered, lagged_Y, on='unit_id', how='left')

    # Drop redundant columns (keep X and newly transformed variables)
    columns_to_keep = [col for col in result.columns if 'Y_lag_' in col or col in [
        'unit_id', 'y', 'treatment', 'X'
    ]]
    result = result[columns_to_keep]
    result['id'] = np.arange(1, len(result) + 1)  # Add an id column from 1 to n
    result.drop('unit_id', axis=1, inplace=True)

    return result


In [112]:
wdf[wdf['promotion_type_EDLP']]['week_start_date'].value_counts()

week_start_date
2024-12-23    247
2024-09-16    247
2024-10-21    247
2024-11-11    246
2024-10-07    246
             ... 
2023-10-16    160
2023-11-06    160
2023-10-23    158
2023-10-30    158
2023-10-09    151
Name: count, Length: 108, dtype: int64

In [113]:
wdf[wdf['week_start_date'] == '2023-10-09']

,unit_id,week_start_date,promotion_type_EDLP,brand,log_qty
108,US04_6902037_5571200,2023-10-09,False,0,0.693147
324,US04_6902037_77800819,2023-10-09,False,0,0.693147
540,US04_6902037_1566500023,2023-10-09,False,PIRATES BRAND PUFFS & CURLS,0.693147
756,US04_6902037_1566500033,2023-10-09,False,PIRATES BRAND PUFFS & CURLS,0.693147
972,US04_6902037_1566525123,2023-10-09,True,PIRATES BRAND PUFFS & CURLS,0.693147
...,...,...,...,...,...
362340,USBX_6902053_5025100402,2023-10-09,False,SKINNYPOP POPCORN,0.693147
362556,USBX_6902053_5025100408,2023-10-09,False,SKINNYPOP POPCORN,0.693147
362772,USBX_6902053_5025100417,2023-10-09,False,SKINNYPOP POPCORN,0.693147
362988,USBX_6902053_5025100420,2023-10-09,False,SKINNYPOP POPCORN,0.693147


In [114]:
# Transform data
transformed_df = transform_to_cross_sectional(wdf, relative_week='2023-10-09')

In [115]:
transformed_df

,treatment,y,Y_lag_2021-09-13 00:00:00,Y_lag_2021-09-20 00:00:00,Y_lag_2021-09-27 00:00:00,Y_lag_2021-10-04 00:00:00,Y_lag_2021-10-11 00:00:00,Y_lag_2021-10-18 00:00:00,Y_lag_2021-10-25 00:00:00,Y_lag_2021-11-01 00:00:00,...,Y_lag_2023-08-14 00:00:00,Y_lag_2023-08-21 00:00:00,Y_lag_2023-08-28 00:00:00,Y_lag_2023-09-04 00:00:00,Y_lag_2023-09-11 00:00:00,Y_lag_2023-09-18 00:00:00,Y_lag_2023-09-25 00:00:00,Y_lag_2023-10-02 00:00:00,Y_lag_2023-10-09 00:00:00,id
0,False,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,1
1,False,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,2
2,False,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,9.943765,10.734264,9.732818,9.078179,11.507129,5.891644,6.582025,0.693147,0.693147,3
3,False,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,4
4,True,0.693147,10.174354,9.562545,10.882227,9.929253,10.176640,9.821301,10.070780,10.068239,...,7.621685,5.488938,6.045005,7.650645,7.923710,8.594525,9.036225,0.693147,0.693147,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,False,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,1678
1678,False,0.693147,0.693147,0.693147,5.669881,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,1679
1679,False,0.693147,0.693147,0.693147,4.983607,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,1680
1680,False,0.693147,0.693147,0.693147,5.267858,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,1681


In [116]:
transformed_df['treatment'].value_counts()

treatment
False    1531
True      151
Name: count, dtype: int64

## Call the DSI model

In [126]:
import sys
from pathlib import Path

# Add the 'src' folder to the Python path
sys.path.append(str(Path().resolve().parent.parent / 'src'))

from utils import byte_conversion,backward_regression
from DSI_Models import DSIModel

In [129]:
# Save interium files
output_path="./output/"
temp_path="./temp/"

os.makedirs(output_path, exist_ok=True)
os.makedirs(temp_path, exist_ok=True)

In [128]:
# Configure X covariates
pattern = 'Y_lag_.*'
mask = transformed_df.columns.str.contains(pattern)

X = transformed_df.loc[:,mask]
all_feature_list = X.columns.tolist()
all_feature_list

['Y_lag_2021-09-13 00:00:00',
 'Y_lag_2021-09-20 00:00:00',
 'Y_lag_2021-09-27 00:00:00',
 'Y_lag_2021-10-04 00:00:00',
 'Y_lag_2021-10-11 00:00:00',
 'Y_lag_2021-10-18 00:00:00',
 'Y_lag_2021-10-25 00:00:00',
 'Y_lag_2021-11-01 00:00:00',
 'Y_lag_2021-11-08 00:00:00',
 'Y_lag_2021-11-15 00:00:00',
 'Y_lag_2021-11-22 00:00:00',
 'Y_lag_2021-11-29 00:00:00',
 'Y_lag_2021-12-06 00:00:00',
 'Y_lag_2021-12-13 00:00:00',
 'Y_lag_2021-12-20 00:00:00',
 'Y_lag_2021-12-27 00:00:00',
 'Y_lag_2022-01-03 00:00:00',
 'Y_lag_2022-01-10 00:00:00',
 'Y_lag_2022-01-17 00:00:00',
 'Y_lag_2022-01-24 00:00:00',
 'Y_lag_2022-01-31 00:00:00',
 'Y_lag_2022-02-07 00:00:00',
 'Y_lag_2022-02-14 00:00:00',
 'Y_lag_2022-02-21 00:00:00',
 'Y_lag_2022-02-28 00:00:00',
 'Y_lag_2022-03-07 00:00:00',
 'Y_lag_2022-03-14 00:00:00',
 'Y_lag_2022-03-21 00:00:00',
 'Y_lag_2022-03-28 00:00:00',
 'Y_lag_2022-04-04 00:00:00',
 'Y_lag_2022-04-11 00:00:00',
 'Y_lag_2022-04-18 00:00:00',
 'Y_lag_2022-04-25 00:00:00',
 'Y_lag_20

In [130]:
# call the DSI model
src_path = str(Path().resolve().parent.parent / 'src')
config_path = src_path + "/config.yaml"

obj_DSI = DSIModel(transformed_df, all_feature_list, config_path, verbose = False)

# obtain theta and se estimates
obj_DSI.fit(save_results = False)
summary = obj_DSI.summary

print(summary)

       coef   std err
0 -0.239316  0.626037
